# Dallas Agent Workshop - Colab Quickstart

This notebook runs the workshop repo in Google Colab.

Steps:
1. Clone the repo + install dependencies
2. Set `OPENROUTER_API_KEY` (and optionally `TAVILY_API_KEY`)
3. Run the preflight + demo calls


## Agenda

- 5:30 - Check-in, food, networking (30)
- 6:00 - Why Agents? (Motivation & Framing) (5)
- 6:05 - Core Concepts & Architectures (5)
- 6:10 - Setup & Environment (30)
- 6:40 - Live Code Walkthrough (20)
- 7:00 - Hands-On Build Session (40)
- 7:40 - Engineering Discipline for Agents (10)
- 7:50 - Next: Virtual sessions, submitting PRs (10)
- 8:00 - Curated Resources


## Why Agents? (Motivation & Framing)

An agent is a loop that can plan, use tools, and iterate toward a goal. In this workshop, the agent can generate Python, execute it, and self-correct based on stdout/stderr.


## Core Concepts & Architectures

We use a simple workflow: `plan -> exec -> (fix -> exec)* -> finish`.

Key ideas:
- State: task, generated code, last run result
- Tools: controlled actions (Python execution, web search)
- Guardrails: timeouts, blocked imports, output requirements


In [ ]:
!git clone https://github.com/jiankunliu-ai/dallas-ai-agent-workshop.git
%cd dallas-ai-agent-workshop
!pip -q install -r requirements.txt


## Set API Keys

Use Colab's prompt to avoid hardcoding secrets into the notebook.

If you get a 401 later, restart the runtime and rerun this cell.


In [ ]:
import os
from getpass import getpass

openrouter_key = getpass('OPENROUTER_API_KEY: ').strip()
if not openrouter_key:
    raise RuntimeError('OPENROUTER_API_KEY is required')

os.environ['OPENROUTER_API_KEY'] = openrouter_key
os.environ['OPENROUTER_MODEL'] = 'arcee-ai/trinity-large-preview:free'

tavily_key = getpass('TAVILY_API_KEY (optional, press enter to skip): ').strip()
if tavily_key:
    os.environ['TAVILY_API_KEY'] = tavily_key


In [ ]:
!python test_model.py


In [ ]:
from agent_lib import run_task

task = "Write a Python function to compute Fibonacci(n) efficiently and print Fibonacci(35)."
result = run_task(task)

result['last_run']


In [ ]:
from agent_lib import run_task

tasks = [
    "Parse this CSV string and compute the average of the 'latency_ms' column:\n\nts,latency_ms\n1,120\n2,110\n3,130\n4,90\n",
    "Implement rolling z-score anomaly score for this list and print the top 3 most anomalous points: [10,11,9,10,10,200,11,10,9,10]",
    "Given a list of (user_id, event_time, event_type), compute per-user session counts (30-min gap) and print a dict."
]

for t in tasks:
    print('\n' + '='*80)
    print('TASK:', t)
    out = run_task(t)
    print('OK:', out['last_run']['ok'])
    print('STDOUT:\n', out['last_run']['stdout'])
    if not out['last_run']['ok']:
        print('STDERR:\n', out['last_run']['stderr'])


## Engineering Discipline For Agents

- Manage state explicitly (what the agent knows between steps).
- Manage context (keep prompts structured and small; summarize).
- Token budgeting (avoid dumping large data into the model).
- Governance/safety (limit tools/permissions; log tool calls).


## Next: Virtual Sessions + Submitting PRs

Stretch goals for follow-up sessions:
- More tools with safety boundaries
- Multiple collaborating agents (planner + specialists)
- Lightweight evaluations (golden prompts, success criteria)

Improvements are welcome via PRs to this repo.


## Curated Resources

- Prompting best practices (Claude): https://platform.claude.com/docs/en/build-with-claude/prompt-engineering/claude-prompting-best-practices
- LangGraph docs: https://langchain-ai.github.io/langgraph/
- OpenRouter docs: https://openrouter.ai/docs


In [ ]:
from research_agent import run_research

question = "What are the top 3 AI chip companies in 2024 and what's their competitive advantage?"

print(f"RESEARCH QUESTION:\n{question}\n")

result = run_research(question)

print('\n' + '='*60)
print('FINAL REPORT:')
print('='*60)
print(result["report"])
